# Wavefront and WavefrontK

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pmd_beamphysics.wavefront.wavefront import Wavefront
from pmd_beamphysics.wavefront.propagators import drift_wavefront
from pmd_beamphysics.wavefront.gaussian import add_gaussian

import numpy as np

import matplotlib.pyplot as plt

In [ ]:
W = Wavefront(
    Ex=np.zeros((11, 11, 2)).astype(complex),
    # Ex = np.random.rand(2,2,2).astype(complex),
    dx=1,
    dy=1,
    dz=0.1,
)
W.Ex[5, 5, :] = 1


W.xmin, W.xmax, W.xvec

W.kxmin, W.kxmax, W.dkx, W.kzvec.min() == W.kzmin, W.kzvec.max() == W.kzmax

W2 = W.to_kspace()
W3 = W2.to_rspace()
#
#
assert np.allclose(W3.Ex, W.Ex)
#
e1 = W.energy
e2 = W2.energy
e3 = W3.energy
e1, e2, e3, e1 / e2

In [ ]:
W.plot(logscale=True)

In [ ]:
W = Wavefront(Ex=np.zeros((11, 11, 2)))
W.Ex[5, 5, :] = 1

W.plot()

In [ ]:
W2 = drift_wavefront(W.pad(1000, 1000, 0), 1000)

W2.plot()

# Gaussian pulse 

In [ ]:
W = Wavefront(
    Ex=np.zeros((101, 101, 51)),
    dx=10e-6,
    dy=10e-6,
    dz=10e-6,
    wavelength=1e-9,
)


zR = 10
add_gaussian(W, zR=zR, energy=1.2345, x0=0)

In [ ]:
W.sigma_x

In [ ]:
W.energy

In [ ]:
W.plot()

In [ ]:
%%time
Zlist = np.linspace(0, 10, 20)
Wlist = [drift_wavefront(W, z) for z in Zlist]

sizes = np.array([w.sigma_x for w in Wlist])
sizes

In [ ]:
sigma_x0 = W.sigma_x

expected_w = sigma_x0 * np.sqrt(1 + (Zlist / zR) ** 2)

In [ ]:
fig, ax = plt.subplots()
ax.plot(Zlist, 1e6 * expected_w, label="expected")
ax.plot(Zlist, 1e6 * sizes, "--", label="propagated")

ax.set_xlabel(r"$z$ (m)")

ax.set_ylabel(r"$\sigma_x$ (µm)")
plt.legend()

# M^2 fit

The beam size squared equation is given by:

$$
w^2(z) = w_0^2 \left[ 1 + \left(\frac{M^2 \lambda}{\pi w_0^2} (z - z_0) \right)^2 \right]
$$

Where:
- $w(z)$: Beam radius at position \(z\).
- $w_0$: Beam waist radius (minimum beam size).
- $z_0$: Position of the beam waist.
- $z$: Position along the propagation axis.
- $M^2$: Beam quality factor.
- $\lambda$: Wavelength of the light.

In this equation:
- At $z = z_0$, the beam size is at its minimum: $w^2(z_0) = w_0^2$.
- As $z$ increases or decreases from $z_0$, the beam size squared grows quadratically, scaled by the $M^2$ parameter.


In [ ]:
z = Zlist
w2 = (2 * expected_w) ** 2
wavelength = W.wavelength

In [ ]:
from scipy.optimize import curve_fit


# Define the beam size squared function
def beam_size_squared(z, w0, z0, M2, wavelength=wavelength):
    k = M2 * wavelength / (np.pi * w0**2)  # Divergence coefficient
    return w0**2 * (1 + (k * (z - z0)) ** 2)


# Initial guesses for w0, z0, and M2
initial_guess = [1e-3, 0.0, 1.0]  # Adjust as needed for your system

# Curve fitting
popt, pcov = curve_fit(beam_size_squared, z, w2, p0=initial_guess)

# Extract fitted parameters
w0_fit, z0_fit, M2_fit = popt

# Print results
print(f"Fitted w0 (beam waist): {w0_fit:.6e} m")
print(f"Fitted z0 (waist position): {z0_fit:.6e} m")
print(f"Fitted M^2 (beam quality): {M2_fit:.6f}")

# Plot the results
z_fit = np.linspace(min(z), max(z), 500)
w2_fit = beam_size_squared(z_fit, *popt)

plt.figure(figsize=(8, 6))
plt.scatter(z, w2, label="Data", color="blue", marker="o")
plt.plot(z_fit, w2_fit, label="Fit", color="red")
plt.xlabel("z (m)")
plt.ylabel("Beam Size Squared (w²) (m²)")
plt.legend()
plt.title("Beam Size Squared vs. z")
plt.grid()
plt.show()

In [ ]:
w0_fit / (W.sigma_x * 2)